In [ ]:
# Import necessary packages
%matplotlib inline
import matplotlib.pyplot as plt
import statistics as stats
import numpy as np
import glob
from astropy.io import fits
from astropy.stats import median_absolute_deviation as medabsdev
import time

In [ ]:
# Sigma clipping by standard deviation
def sigma_clip_std(data, alpha):
    """
    Function that performs sigma clipping on a set of data
    1) Take in data
    2) Separate and evaluate the values at each pixel
    3) Perform the sigma clip at each pixel
    4) Continue until the last pixel has been evaulated
    5) Output the clipped array for analysis
    """
    '''
    Parameters of the function
    1) Take in the given data
    2) define necessary variables for the main function
    3) Timer for possible optimization
    '''
    # Parameters
    data      = np.array(data)                  # Data to be clipped
    alpha     = alpha                           # Number of stddev to perform clip
    shape     = np.shape(data)                  # Dimensions of given array
    data_clip = np.empty([shape[1],shape[2]])   # Array for clipped data
    pixel     = []                              # Empty array for pixel values
    median    = 0                               # Median of the pixel data
    std       = 0                               # Standard deviation of pixel data
    newstd    = 0                               # Standard deviation after clipping
    t_o       = time.clock()                    # Timer for performance
    
    '''
    Main body for the sigma clip function
    1) Iterate through the array
    2) Isolate the pixel values
    3) Perform the sigma clip on pixel data
        3.1) Calculate Median and Standard Deviation
        3.2) Delete all values that are larger/smaller than median +/- alpha*std
        3.3) Repeat until desired iteration has occurred
        3.4) Average pixel values and place them in new array
    4) Repeat until the last pixel
    5) Provide the clipped array as output
    '''
    # Iterate through data
    for i in range(shape[1]):
        for j in range(shape[2]):
            pixel = data[:,i,j]
            while std != newstd:
                k = len(pixel) - 1
                std = np.std(pixel)
                median = np.median(pixel)
                while k >= 0:
                    if (abs(pixel[k] - median) / std) > alpha:
                        pixel = np.delete(pixel, k)
                    k -= 1
                newstd = np.std(pixel)
            data_clip[i][j] = sum(pixel) / len(pixel)
            
    # Print elapsed time
    t_f = time.clock()
    print("Time Elapsed =", t_f-t_o, "seconds")
    
    # Return the clipped array
    return(data_clip)

In [ ]:
def sigma_clip_mad(data, alpha, iterate):
    """
    Function that performs sigma clipping on a set of data
    1) Take in data
    2) Separate and evaluate the values at each pixel
    3) Perform the sigma clip at each pixel
    4) Continue until the last pixel has been evaulated
    5) Output the clipped array for analysis
    """
    '''
    Parameters of the function
    1) Take in the given data
    2) define necessary variables for the main function
    3) Timer for possible optimization
    '''
    # Parameters
    data        = np.array(data)                  # Data to be clipped
    alpha       = alpha                           # Number of stddev to perform clip
    iterate     = iterate                         # Number of iterations for clipping process
    shape       = np.shape(data)                  # Dimensions of given array
    data_clip   = np.empty([shape[1],shape[2]])   # Array for clipped data
    data_uncert = np.empty([shape[1],shape[2]])   # Array for uncertainty
    pixel       = []                              # Empty array for pixel values
    median      = 0                               # Median of the pixel data
    mad         = 0                               # Median absolute deviation of pixel data
    newmad      = 0                               # Median absolute deviation after clipping
    t_o         = time.clock()                    # Timer for performance
    
    '''
    Main body for the sigma clip function
    1) Iterate through the array
    2) Isolate the pixel values
    3) Perform the sigma clip on pixel data
        3.1) Calculate Median and Median Absolute Deviation
        3.2) Delete all values that are larger/smaller than median +/- alpha*mad
        3.3) Repeat until desired iteration has occurred
        3.4) Average pixel values and place them in new array
    4) Repeat until the last pixel
    5) Provide the clipped array as output
    '''
    # Iterate through data
    for i in range(shape[1]):
        for j in range(shape[2]):
            pixel = data[:,i,j]
            index = 0
            while index < 3:
                p = len(pixel) - 1
                median = np.median(pixel)
                mad = 1.4826 * medabsdev(pixel)
                if mad == 0:
                    break
                while p >= 0:
                    if (abs(pixel[p] - median) / mad) > alpha:
                        pixel = np.delete(pixel, p)
                    p -= 1
                index += 1
            data_clip[i][j] = sum(pixel) / len(pixel)
            
    # Print elapsed time
    t_f = time.clock()
    print("Time Elapsed =", t_f-t_o, "seconds")
    
    # Return the clipped array
    return(data_clip)

In [ ]:
# Generate a list of the image data
img_list = glob.glob('')
image_concat = np.array([fits.getdata(image) for image in img_list])

'''
Uncomment and copy for separate sets of data
    - Replace n with the set number for organization
'''
# img_listn = glob.glob('')
# image_concatn = [fits.getdata(image) for image in img_listn]

'''
Uncomment if shifting is necessary
    - n is the image set number
    - m is the number of pixels to shift
    - axis=0 is a vertical shift
    - axis=1 is a horizontal shift
'''
# for i in range(len(image_concatn)):
#     image_concatn[i] = np.roll(image_concatn[i], m, axis=0)
#     image_concatn[i] = np.roll(image_concatn[i], m, axis=1)

'''
Uncomment to combine the separated image set
'''
# image_combined = image_concatn + image_concatn

In [ ]:
# Combine the images without clipping
'''
Can replace average with sum if preferred
'''
image = np.average(image_concat, axis=0)
plt.imshow(image, cmap='gray', vmin=0, vmax=0.5)
plt.colorbar()

# Create a file for the image
outfile = '.fits'
hdu = fits.PrimaryHDU(image)
hdu.writeto(outfile, overwrite=True)

In [ ]:
# Combine the images after clipping
'''
The output of sigma_clip will be the average of the remaining 
pixel values after the clipping process
'''
image_clip = sigma_clip_std(image_concat, alpha)
plt.imshow(image_clip, cmap='gray', vmin=0, vmax=0.5)
plt.colorbar()

'''
Uncomment if clipping by MAD is preferred
'''
# image_clip = sigma_clip_mad(image_concat, alpha, iterate)
# plt.imshow(image_clip, cmap='gray', vmin=0, vmax=0.5)
# plt.colorbar()

# Create a file for the image
outfile = '.fits'
hdu = fits.PrimaryHDU(image_clip)
hdu.writeto(outfile, overwrite=True)